1. Install Required Packages

In [31]:
!pip install replicate duckduckgo-search gradio


2. Import Libraries and Set Up Clients

In [32]:
import replicate
from duckduckgo_search import DDGS

# Paste your Replicate API key here (from Replicate dashboard)
REPLICATE_API_TOKEN = "r8_7DKzIp8QC452KxIBwZ0bzZ07ujHZj0O4ItIeK"
client = replicate.Client(api_token=REPLICATE_API_TOKEN)


3. Optional: Search Function


In [33]:
def search_duckduckgo(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join([f"- {r['title']}: {r['href']}" for r in results])


4. LLM Query Function (IBM Granite 3‑3.8B)

In [34]:
def query_replicate(prompt):
    # Force IBM Granite 3.3-8B Instruct model usage
    output = client.run(
        "ibm-granite/granite-3.3-8b-instruct",
        input={"prompt": prompt, "max_new_tokens": 300}
    )
    return "".join(output)


5.Define Agent Roles

a. Crop Advisor Agent

In [35]:
def crop_advisor_agent(prompt):
    system_prompt = (
        "You are a smart agricultural crop advisor. Provide detailed crop selection "
        "and crop health advice based on soil, weather, and market trends."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


b. Weather & Irrigation Agent

In [36]:
def weather_irrigation_agent(prompt):
    system_prompt = (
        "You are an AI weather and irrigation expert. Suggest irrigation schedules, "
        "weather precaution alerts, and field preparation advice based on current and forecasted conditions."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


c. Pest and Disease Detection Agent

In [37]:
def pest_disease_agent(prompt):
    system_prompt = (
        "You are an agriculture diagnostic AI. Identify likely pests or diseases from "
        "user-provided farm observations and suggest preventive and curative measures."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


d. Price/Market Info Agent (Approximate Values)

In [38]:
def price_agent(prompt):
    system_prompt = (
        "You are an agricultural market analyst. Estimate the approximate or nearest mandi price "
        "for the mentioned crop or vegetable based on recent trends from typical markets. "
        "Include a disclaimer that the value is approximate and varies by region and market conditions."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


6.Routing User Queries

In [39]:
def smart_farming_router(user_prompt):
    crop_keywords = ["crop", "plant", "harvest", "yield", "seed", "fertilizer"]
    weather_keywords = ["weather", "rain", "temperature", "irrigation", "moisture", "water", "climate"]
    pest_keywords = ["pest", "disease", "fungus", "insect", "infestation", "blight", "mite"]
    price_keywords = ["price", "market", "mandi", "vegetable price", "cost", "rate"]

    prompt_lower = user_prompt.lower()

    if any(word in prompt_lower for word in crop_keywords):
        return crop_advisor_agent(user_prompt)
    elif any(word in prompt_lower for word in weather_keywords):
        return weather_irrigation_agent(user_prompt)
    elif any(word in prompt_lower for word in pest_keywords):
        return pest_disease_agent(user_prompt)
    elif any(word in prompt_lower for word in price_keywords):
        return price_agent(user_prompt)
    else:
        return "🌱 Please specify if you want crop advice, weather/irrigation help, pest/disease diagnosis, or market price info."


7. Gradio App Interface

In [40]:
import gradio as gr

def gradio_interface(user_prompt):
    return smart_farming_router(user_prompt)

app = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, label="Enter your farming question (crops, weather, pests, prices…)"),
    outputs=gr.Textbox(label="Smart Farming AI Response"),
    title="Smart Farming AI Agent",
    description="An AI agent that provides crop advice, weather & irrigation tips, pest diagnosis, and approximate market prices. Powered by IBM Granite model via Replicate."
)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c922c883ff22cf5ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
